In [1]:
%cd ../

/home/users/dmoreno2016/VT_Model_for_LightCurves_Classification


/home/users/dmoreno2016/miniconda3/envs/visualtrans_git/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [132]:
import pandas as pd
import yaml

from src.data.processing.get_data import get_dataset

def load_yaml(path):
    with open(path, 'r') as file:
        args = yaml.load(file, Loader=yaml.FullLoader)
    return args

In [133]:
partition = pd.read_parquet('./data/lightcurves/macho/ASTROMER_partition/partitions_v1.parquet')

display(partition)

,lcid,label,subset,fold,spc,class_name
0,6.6693.401,4,val,0,500,RRab
1,14.8613.692,4,val,0,500,RRab
2,15.10793.263,4,val,0,500,RRab
3,80.6592.753,4,val,0,500,RRab
4,80.7196.1648,4,val,0,500,RRab
...,...,...,...,...,...,...
81937,13.6326.2765,5,train,1,50,RRc
81938,2.4904.887,5,train,1,50,RRc
81939,6.6572.919,5,train,1,50,RRc
81940,79.5134.853,5,train,1,50,RRc


In [134]:
partition.dtypes

lcid          object
label          int32
subset        object
fold           int64
spc           object
class_name    object
dtype: object

In [135]:
datasets_config = load_yaml(path='configs/datasets_config.yaml')

name_dataset = 'macho_multiband'
macho_multiband_config = datasets_config[name_dataset]
macho_dataset = get_dataset(macho_multiband_config, name_dataset, debug=False)
macho_dataset

,mjd,mag,err,lcid,band
0,48823.66797,-8.606,0.014,5.4408.1653,1
1,48824.67578,-8.616,0.007,5.4408.1653,1
2,48825.69531,-8.607,0.005,5.4408.1653,1
3,48826.68359,-8.609,0.007,5.4408.1653,1
4,48828.63672,-8.616,0.011,5.4408.1653,1
...,...,...,...,...,...
29217169,51532.67188,-6.825,0.038,79.5378.411,0
29217170,51535.63281,-6.789,0.030,79.5378.411,0
29217171,51541.46484,-6.797,0.036,79.5378.411,0
29217172,51542.73438,-6.804,0.049,79.5378.411,0


In [136]:
bright_biased_df_final = {}
bright_biased_sample = {}
test_oids = {}

for spc in [20, 500]:
    for fold in range(3):
        # Only use examples marked as spc=='all' to define the full pool
        filtered = partition[(partition.fold == fold) & (partition.spc == 'all')]

        # Separate labeled and test sets
        class_oids = filtered[filtered.subset.isin(['val', 'train'])].copy()
        test_set = filtered[filtered.subset == 'test'].copy()

        # Store test set, but update spc so it's consistent with the current sampling
        test_set["spc"] = str(spc)
        test_oids[f'fold_{fold}_spc_{spc}'] = test_set

        # Get brightness info for sorting
        macho_train_val = macho_dataset[macho_dataset.lcid.isin(class_oids['lcid'].values)]
        median_mag_df = (
            macho_train_val.groupby('lcid')['mag']
            .median()
            .reset_index()
            .rename(columns={'mag': 'median_mag'})
        )

        # Merge median mag with labeled
        labeled_brightness_df = class_oids.merge(median_mag_df, on='lcid')

        # Sort and take top N=spc per class
        labeled_brightness_df = labeled_brightness_df.sort_values(['class_name', 'median_mag'])
        bright_sample = (
            labeled_brightness_df.groupby('class_name', group_keys=False)
            .apply(lambda x: x.head(spc))
            .reset_index(drop=True)
        )

        # Overwrite 'spc' column so it's no longer "all"
        bright_sample['spc'] = str(spc)
        bright_biased_sample[f'fold_{fold}_spc_{spc}'] = bright_sample

        # Also store the full LC info for the selected bright sample
        bright_biased_df_final[f'fold_{fold}_spc_{spc}'] = macho_train_val[
            macho_train_val['lcid'].isin(bright_sample['lcid'])
        ].copy()

/tmp/ipykernel_23182/4133543766.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.head(spc))
/tmp/ipykernel_23182/4133543766.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.head(spc))
/tmp/ipykernel_23182/4133543766.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas t

In [137]:
bright_biased_sample.keys()

dict_keys(['fold_0_spc_20', 'fold_1_spc_20', 'fold_2_spc_20', 'fold_0_spc_500', 'fold_1_spc_500', 'fold_2_spc_500'])

In [138]:
bright_biased_sample['fold_0_spc_500']

,lcid,label,subset,fold,spc,class_name,median_mag
0,1.3810.11,0,train,0,500,Cep_0,-10.2760
1,1.3686.7,0,train,0,500,Cep_0,-10.1980
2,24.2859.10,0,train,0,500,Cep_0,-10.1560
3,79.4898.12,0,train,0,500,Cep_0,-10.1460
4,17.2947.9,0,train,0,500,Cep_0,-10.1455
...,...,...,...,...,...,...,...
2995,24.3581.2110,5,train,0,500,RRc,-5.0420
2996,7.7178.1252,5,train,0,500,RRc,-5.0420
2997,9.4873.519,5,train,0,500,RRc,-5.0420
2998,22.5468.258,5,train,0,500,RRc,-5.0420


In [139]:
for spc in [20, 500]:
    for fold in range(3):
        print(bright_biased_sample[f'fold_{fold}_spc_{spc}'].class_name.value_counts())

class_name
Cep_0    20
Cep_1    20
EC       20
LPV      20
RRab     20
RRc      20
Name: count, dtype: int64
class_name
Cep_0    20
Cep_1    20
EC       20
LPV      20
RRab     20
RRc      20
Name: count, dtype: int64
class_name
Cep_0    20
Cep_1    20
EC       20
LPV      20
RRab     20
RRc      20
Name: count, dtype: int64
class_name
Cep_0    500
Cep_1    500
EC       500
LPV      500
RRab     500
RRc      500
Name: count, dtype: int64
class_name
Cep_0    500
Cep_1    500
EC       500
LPV      500
RRab     500
RRc      500
Name: count, dtype: int64
class_name
Cep_0    500
Cep_1    500
EC       500
LPV      500
RRab     500
RRc      500
Name: count, dtype: int64


In [140]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Create list to collect all rows from all folds and SPCs
all_rows = []

for spc in [20, 500]:  # or whatever values you're using
    for fold in range(3):
        # Get the bright sample for this fold and spc
        df = bright_biased_sample[f'fold_{fold}_spc_{spc}'].copy()

        # Drop unwanted columns if they exist
        df = df.drop(columns=[col for col in ['subset', 'fall', 'median_mag'] if col in df.columns])

        # Stratified split
        train_df, val_df = train_test_split(
            df,
            test_size=0.2,
            stratify=df["class_name"],
            random_state=42
        )

        # Add 'subset' and 'fold' columns
        train_df["subset"] = "train"
        val_df["subset"] = "val"
        train_df["fold"] = fold
        val_df["fold"] = fold

        # Retrieve corresponding test set and ensure 'spc' is correct
        test_df = test_oids[f'fold_{fold}_spc_{spc}'].copy()
        test_df = test_df.drop(columns=[col for col in ['fall', 'median_mag'] if col in test_df.columns])
        test_df["subset"] = "test"
        test_df["fold"] = fold

        # Collect all splits
        all_rows += [train_df, val_df, test_df]

# Combine all data into one DataFrame
final_df = pd.concat(all_rows, ignore_index=True)

# Reorder columns as needed
desired_order = ['lcid', 'label', 'subset', 'fold', 'spc', 'class_name']
final_df = final_df[[col for col in desired_order if col in final_df.columns]]

# Show result
final_df


,lcid,label,subset,fold,spc,class_name
0,1.4297.13,3,train,0,20,LPV
1,79.4661.1703,0,train,0,20,Cep_0
2,14.8983.189,5,train,0,20,RRc
3,5.5498.4779,1,train,0,20,Cep_1
4,78.6706.1641,4,train,0,20,RRab
...,...,...,...,...,...,...
12955,2.4904.887,5,test,2,500,RRc
12956,18.3211.679,5,test,2,500,RRc
12957,7.7293.755,5,test,2,500,RRc
12958,13.6808.718,5,test,2,500,RRc


In [141]:
final_df.groupby(['fold', 'subset', 'spc'])['lcid'].nunique()

fold  subset  spc
0     test    20      600
              500     600
      train   20       96
              500    2400
      val     20       24
              500     600
1     test    20      600
              500     600
      train   20       96
              500    2400
      val     20       24
              500     600
2     test    20      600
              500     600
      train   20       96
              500    2400
      val     20       24
              500     600
Name: lcid, dtype: int64

In [142]:
final_df.groupby(['fold', 'subset', 'spc', 'class_name'])['lcid'].nunique()

fold  subset  spc  class_name
0     test    20   Cep_0         100
                   Cep_1         100
                   EC            100
                   LPV           100
                   RRab          100
                                ... 
2     val     500  Cep_1         100
                   EC            100
                   LPV           100
                   RRab          100
                   RRc           100
Name: lcid, Length: 108, dtype: int64

In [143]:
final_df.to_parquet('data/lightcurves/macho/bright_biased_partition/partitions_v1.parquet')

In [144]:
final_df

,lcid,label,subset,fold,spc,class_name
0,1.4297.13,3,train,0,20,LPV
1,79.4661.1703,0,train,0,20,Cep_0
2,14.8983.189,5,train,0,20,RRc
3,5.5498.4779,1,train,0,20,Cep_1
4,78.6706.1641,4,train,0,20,RRab
...,...,...,...,...,...,...
12955,2.4904.887,5,test,2,500,RRc
12956,18.3211.679,5,test,2,500,RRc
12957,7.7293.755,5,test,2,500,RRc
12958,13.6808.718,5,test,2,500,RRc


In [146]:
final_df.dtypes

lcid          object
label          int32
subset        object
fold           int64
spc           object
class_name    object
dtype: object